In [1]:
import os
import json
import re

import numpy as np
import pandas as pd
import torch

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [29]:
from violet.utils.dataloaders import listfiles
from violet.utils.st import predict_he_tiles, predict_visium, predict_svs

In [8]:
img_dir = '/home/estorrs/violet/data/st/human_he_06252021/train/HT206B1_H8_U2/'
weights = '/home/estorrs/violet/sandbox/runs/test_run_brca_06242021/checkpoints/final.pt'
summary = '/home/estorrs/violet/sandbox/runs/test_run_brca_06242021/summary.json'

#### from image directory

In [18]:
df = predict_he_tiles(img_dir, weights, summary, gpu=False)
df

/home/estorrs/miniconda3/envs/violet/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


,ESR1,PGR,ERBB2,MKI67,TOP2A,CD3G,CD4,CD8A,KIT,EPCAM,...,BGN,FAP,SPARC,ITGAX,LYZ,CD68,CD14,SDC1,PECAM1,IL7R
HT206B1_H8_U2_AAACAACGAATAGTTC-1,-0.164906,0.175787,0.621281,0.213999,0.076712,0.140059,0.573150,0.179334,0.213385,0.736988,...,1.896825,0.045795,1.614560,0.044590,0.495791,0.974455,1.180156,0.656987,0.498567,0.354398
HT206B1_H8_U2_AAACAAGTATCTCCCA-1,0.083358,-0.067511,0.164565,0.407814,0.379001,0.104328,0.428431,0.037954,0.014664,0.198301,...,1.599838,0.259282,1.708095,0.042593,0.723933,0.647760,0.972288,0.355240,0.583958,0.146070
HT206B1_H8_U2_AAACAATCTACTAGCA-1,0.106662,-0.094008,0.571161,-0.041895,0.366605,-0.056045,0.480472,0.179056,0.322798,0.106388,...,1.395966,0.535094,1.995656,0.393462,0.594866,0.902277,0.512430,0.749007,0.490630,0.375575
HT206B1_H8_U2_AAACACCAATAACTGC-1,-0.045001,-0.027916,-0.016639,0.302272,0.170497,0.269644,0.525026,0.051995,0.148004,0.090111,...,1.449061,-0.062446,1.518379,-0.065594,0.653827,0.864539,1.006046,0.744373,0.524632,0.237762
HT206B1_H8_U2_AAACAGAGCGACTCCT-1,0.102596,-0.154902,0.552773,0.332097,0.400498,0.184301,0.630479,0.378817,0.193894,0.345706,...,1.699734,0.162902,1.897868,0.390524,1.412358,0.964725,1.131966,0.708836,0.606410,0.639564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
<pad_14>,-0.164906,0.175787,0.621281,0.213999,0.076712,0.140059,0.573150,0.179334,0.213385,0.736988,...,1.896825,0.045795,1.614560,0.044590,0.495791,0.974455,1.180156,0.656987,0.498567,0.354398
<pad_15>,-0.164906,0.175787,0.621281,0.213999,0.076712,0.140059,0.573150,0.179334,0.213385,0.736988,...,1.896825,0.045795,1.614560,0.044590,0.495791,0.974455,1.180156,0.656987,0.498567,0.354398
<pad_16>,-0.164906,0.175787,0.621281,0.213999,0.076712,0.140059,0.573150,0.179334,0.213385,0.736988,...,1.896825,0.045795,1.614560,0.044590,0.495791,0.974455,1.180156,0.656987,0.498567,0.354398
<pad_17>,-0.164906,0.175787,0.621281,0.213999,0.076712,0.140059,0.573150,0.179334,0.213385,0.736988,...,1.896825,0.045795,1.614560,0.044590,0.495791,0.974455,1.180156,0.656987,0.498567,0.354398


#### from visium

In [21]:
filemap = pd.read_csv('/home/estorrs/spatial-analysis/data/sample_map.txt', sep='\t')
filemap = filemap[~pd.isnull(filemap['highres_image'])]
filemap = filemap.set_index('sample_id')
filemap

,spaceranger_output,highres_image,disease
sample_id,,,
HT206B1_H8_U2,/data/spatial_transcriptomics/spaceranger_outp...,/data/spatial_transcriptomics/highres_images/b...,brca
HT206B1_H8_U3,/data/spatial_transcriptomics/spaceranger_outp...,/data/spatial_transcriptomics/highres_images/b...,brca
HT206B1_H8_U4,/data/spatial_transcriptomics/spaceranger_outp...,/data/spatial_transcriptomics/highres_images/b...,brca
HT206B1_H8_U5,/data/spatial_transcriptomics/spaceranger_outp...,/data/spatial_transcriptomics/highres_images/b...,brca
HT235B1_H1_U1,/data/spatial_transcriptomics/spaceranger_outp...,/data/spatial_transcriptomics/highres_images/b...,brca
...,...,...,...
NMK_20201012,/data/spatial_transcriptomics/spaceranger_outp...,/data/spatial_transcriptomics/highres_images/m...,mouse_kidney
AKICL_14w,/data/spatial_transcriptomics/spaceranger_outp...,/data/spatial_transcriptomics/highres_images/m...,mouse_kidney
AKI_M_14w,/data/spatial_transcriptomics/spaceranger_outp...,/data/spatial_transcriptomics/highres_images/m...,mouse_kidney


In [26]:
spatial, tif = (filemap.loc['HT206B1_H8_U2', 'spaceranger_output'],
                filemap.loc['HT206B1_H8_U2', 'highres_image'])
adata = predict_visium(spatial, tif, weights, summary, tmp_dir='../sandbox/tmp',
                      gpu=False)
adata

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


AnnData object with n_obs × n_vars = 3949 × 36601
    obs: 'in_tissue', 'array_row', 'array_col', 'ESR1', 'PGR', 'ERBB2', 'MKI67', 'TOP2A', 'CD3G', 'CD4', 'CD8A', 'KIT', 'EPCAM', 'CDH1', 'BGN', 'FAP', 'SPARC', 'ITGAX', 'LYZ', 'CD68', 'CD14', 'SDC1', 'PECAM1', 'IL7R'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [27]:
adata.obs

,in_tissue,array_row,array_col,ESR1,PGR,ERBB2,MKI67,TOP2A,CD3G,CD4,...,BGN,FAP,SPARC,ITGAX,LYZ,CD68,CD14,SDC1,PECAM1,IL7R
AAACAACGAATAGTTC-1,1,0,16,-0.164906,0.175787,0.621281,0.213999,0.076712,0.140059,0.573150,...,1.896825,0.045795,1.614560,0.044590,0.495791,0.974455,1.180156,0.656987,0.498567,0.354398
AAACAAGTATCTCCCA-1,1,50,102,0.083358,-0.067511,0.164565,0.407814,0.379001,0.104328,0.428431,...,1.599838,0.259282,1.708095,0.042593,0.723933,0.647760,0.972288,0.355240,0.583958,0.146070
AAACAATCTACTAGCA-1,1,3,43,0.106662,-0.094008,0.571161,-0.041895,0.366605,-0.056045,0.480472,...,1.395966,0.535094,1.995656,0.393462,0.594866,0.902277,0.512430,0.749007,0.490630,0.375575
AAACACCAATAACTGC-1,1,59,19,-0.045001,-0.027916,-0.016639,0.302272,0.170497,0.269644,0.525026,...,1.449061,-0.062446,1.518379,-0.065594,0.653827,0.864539,1.006046,0.744373,0.524632,0.237762
AAACAGAGCGACTCCT-1,1,14,94,0.102596,-0.154902,0.552773,0.332097,0.400498,0.184301,0.630479,...,1.699734,0.162902,1.897868,0.390524,1.412358,0.964725,1.131966,0.708836,0.606410,0.639564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTTCACATCCAGG-1,1,58,42,-0.024233,0.054195,0.220355,-0.034888,-0.095982,-0.038603,0.415327,...,1.582199,0.147915,1.130237,-0.160732,0.297704,0.669519,0.941165,0.445176,0.536740,0.118417
TTGTTTCATTAGTCTA-1,1,60,30,0.098096,0.015549,0.328783,0.315241,0.290279,-0.043594,0.433436,...,1.574781,0.056235,1.474961,-0.132179,0.683777,0.789845,0.972755,0.325716,0.689748,0.129485
TTGTTTCCATACAACT-1,1,45,27,0.010711,0.120938,0.273827,0.263793,0.153023,0.039866,0.393255,...,1.653756,0.145023,2.056172,0.125758,0.694948,0.909101,1.023094,0.407638,0.605458,0.115894
TTGTTTGTATTACACG-1,1,73,41,0.014436,0.281017,0.118035,0.359022,0.232103,0.458492,0.389886,...,1.611174,-0.223090,1.581512,-0.013654,0.799818,0.734726,0.907430,0.516598,0.437036,0.303722


#### from svs

In [28]:
svs_fp = '/home/estorrs/he/cptac/sandbox/C3L-00004-21.svs'
res, background_pct = 55., .5

In [30]:
df = predict_svs(svs_fp, weights, summary, tmp_dir='../sandbox/tmp', gpu=False)
df

predicting 13641 tiles.


/home/estorrs/miniconda3/envs/violet/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


,ESR1,PGR,ERBB2,MKI67,TOP2A,CD3G,CD4,CD8A,KIT,EPCAM,...,BGN,FAP,SPARC,ITGAX,LYZ,CD68,CD14,SDC1,PECAM1,IL7R
C3L-00004-21_100_100,-0.145675,0.251186,-0.231279,0.177751,-0.030363,0.179881,0.668892,0.118153,-0.234897,0.137426,...,1.272607,0.008894,1.082158,0.175976,0.104541,0.811972,0.570227,0.803470,0.515431,0.473850
C3L-00004-21_100_101,-0.191286,0.265296,-0.222078,0.174069,0.043737,0.195653,0.664410,0.076009,-0.144072,0.031477,...,1.421658,0.014048,1.073696,0.228566,0.255660,0.839344,0.456989,0.802486,0.535111,0.306627
C3L-00004-21_100_102,-0.135420,0.283270,-0.296114,0.151370,-0.005815,0.140557,0.588284,0.088035,-0.082496,0.119691,...,1.433241,0.025788,1.096340,0.190226,0.202399,0.821412,0.512350,0.789413,0.532001,0.322961
C3L-00004-21_100_103,-0.125835,0.294961,-0.314067,0.224748,0.032731,0.171098,0.528181,0.057264,-0.068869,0.037486,...,1.436893,0.046299,1.131228,0.174282,0.263809,0.829279,0.500823,0.762957,0.591911,0.252112
C3L-00004-21_100_104,-0.108250,0.302449,-0.199812,0.233259,0.040472,0.065165,0.522456,0.035263,0.034478,0.099491,...,1.435920,0.046776,1.221590,0.176898,0.256590,0.761859,0.493739,0.712767,0.612591,0.134497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
<pad_50>,-0.145675,0.251186,-0.231279,0.177751,-0.030363,0.179881,0.668892,0.118153,-0.234897,0.137426,...,1.272607,0.008894,1.082158,0.175976,0.104541,0.811972,0.570227,0.803470,0.515431,0.473850
<pad_51>,-0.145675,0.251186,-0.231279,0.177751,-0.030363,0.179881,0.668892,0.118153,-0.234897,0.137426,...,1.272607,0.008894,1.082158,0.175976,0.104541,0.811972,0.570227,0.803470,0.515431,0.473850
<pad_52>,-0.145675,0.251186,-0.231279,0.177751,-0.030363,0.179881,0.668892,0.118153,-0.234897,0.137426,...,1.272607,0.008894,1.082158,0.175976,0.104541,0.811972,0.570227,0.803470,0.515431,0.473850
<pad_53>,-0.145675,0.251186,-0.231279,0.177751,-0.030363,0.179881,0.668892,0.118153,-0.234897,0.137426,...,1.272607,0.008894,1.082158,0.175976,0.104541,0.811972,0.570227,0.803470,0.515431,0.473850
